In [ ]:
import Pkg
Pkg.activate("../pha")

In [ ]:
using Revise

In [ ]:
using NEOs
using DataFrames
using TaylorIntegration
using PlanetaryEphemeris
using JLD2
using Statistics
using LinearAlgebra
using Dates
using StatsBase
using Gaston
using DelimitedFiles

In [ ]:
dq = set_variables("δx", order=5, numvars=7)

## Load Sun, Earth and Apophis ephemeris

In [ ]:
sol_bwd = JLD2.load("Apophis_bwd.jld2", "asteph")

In [ ]:
sol_fwd = JLD2.load("Apophis_fwd.jld2", "asteph")

In [ ]:
sseph::TaylorInterpolant{Float64,Float64,2} = loadpeeph(NEOs.sseph, sol_bwd.t0+sol_bwd.t[end], sol_fwd.t0+sol_fwd.t[end])
eph_su::TaylorInterpolant{Float64,Float64,2} = selecteph(sseph, su)
eph_ea::TaylorInterpolant{Float64,Float64,2} = selecteph(sseph, ea)

# NEO
# Change t, x, v units, resp., from days, au, au/day to sec, km, km/sec
xva_bwd(et) = auday2kmsec(sol_bwd(et/daysec)[1:6])
xva_fwd(et) = auday2kmsec(sol_fwd(et/daysec)[1:6])
xva(et) = bwdfwdeph(et, sol_bwd, sol_fwd)
# Earth
# Change x, v units, resp., from au, au/day to km, km/sec
xve(et) = auday2kmsec(eph_ea(et/daysec))
# Sun
# Change x, v units, resp., from au, au/day to km, km/sec
xvs(et) = auday2kmsec(eph_su(et/daysec))

In [ ]:
# get optical observations
radec_2004_2020 = read_radec_mpc(joinpath(pkgdir(NEOs), "data", "99942_2004_2020.dat"))
radec_2020_2021 = read_radec_mpc(joinpath(pkgdir(NEOs), "data", "99942_2020_2021.dat"))
radec = vcat(radec_2004_2020,radec_2020_2021)

In [ ]:
# get radar observations
deldop_2005_2013 = read_radar_jpl(joinpath(pkgdir(NEOs), "data", "99942_RADAR_2005_2013.dat"))
deldop_2021 = read_radar_jpl(joinpath(pkgdir(NEOs), "data", "99942_RADAR_2021.dat"))
deldop = vcat(deldop_2005_2013,deldop_2021)

In [ ]:
# Compute optical residuals
@time res_radec_all, w_radec_all = NEOs.residuals(radec; xvs, xve, xva);

In [ ]:
# Compute radar residuals
@time res_del, w_del, res_dop, w_dop = NEOs.residuals(deldop; xvs, xve, xva, niter=10, tord=10);

## Process optical data (filter, weight, debias)

In [ ]:
# filter out biased observations from observatory 217 on 28-Jan-2021
df_radec = DataFrame(radec)
# add residuals and weights to optical astrometry DataFrame
df_radec[!, :res_α] .= res_radec_all[1:round(Int,length(res_radec_all)/2)]
df_radec[!, :res_δ] .= res_radec_all[1+round(Int,length(res_radec_all)/2):end]
df_radec[!, :w_α] .= w_radec_all[1:round(Int,length(res_radec_all)/2)]
df_radec[!, :w_δ] .= w_radec_all[1+round(Int,length(res_radec_all)/2):end]
filter!(
    x->(Date(x.date) != Date(2021, 1, 28)),
    df_radec
)

# read astrometric errors from Tholen et al. (2013)
tho13_errors = readdlm(joinpath(pkgdir(NEOs), "data", "tholenetal2013_opterror.dat"), ',')
# compute weights
w_α_tho13 = 1 ./ (tho13_errors[:,1].^2 .+ tho13_errors[:,3].^2 .+ tho13_errors[:,5].^2)
w_δ_tho13 = 1 ./ (tho13_errors[:,2].^2 .+ tho13_errors[:,4].^2 .+ tho13_errors[:,6].^2)
# Tholen et al. (2013) obs table
radec_tho13 = DataFrame(read_radec_mpc(joinpath(pkgdir(NEOs), "test", "data", "99942_Tholen_etal_2013.dat")))
# vector of RA values from Tholen et al. (2013) observations (used for filtering)
tho13_α = radec_tho13[!,:α]
# set weights in Tholen et al. (2013) astrometry corresponding to associated uncertainties
df_radec[in.(df_radec.α, Ref(tho13_α)),:w_α] = w_α_tho13
df_radec[in.(df_radec.α, Ref(tho13_α)),:w_δ] = w_δ_tho13

# Relaxation factor (account for correlations in optical astrometry data)
# for each observation batch, count the number of observations made in
# the same night by the same observatory
# Ref: Veres et al. (2017)
date_site_v = select(df_radec, :date => ByRow(Date), :observatory)
Nv = [count(x->x.date_Date==i.date_Date && x.observatory==i.observatory, eachrow(date_site_v)) for i in eachrow(date_site_v)]
relax_factor = map(x->x>4.0 ? x/4.0 : 1.0, Nv)
# inflate uncertainties (i.e., relax weights) by relaxation factor
df_radec[!, :w_α] .= (df_radec.w_α)./relax_factor
df_radec[!, :w_δ] .= (df_radec.w_δ)./relax_factor

# update optical residuals and weights
res_radec = vcat(df_radec.res_α, df_radec.res_δ)
w_radec = vcat(df_radec.w_α, df_radec.w_δ)
;

In [ ]:
# check if all RA/Dec observations are CCD:

all(
    select(
        df_radec, 
        :obstech => ByRow(x->x == "C" || x == "X")
        )[!,1]
)

In [ ]:
# there are 512 RA/Dec obs without catalog info

length(df_radec[isunknown.(df_radec.catalogue),:catalogue])

In [ ]:
# Construct vector of residuals and weights

res = vcat(res_radec, res_del, res_dop)
w = vcat(w_radec, w_del, w_dop)
;

In [ ]:
### Perform 7-DOF orbital fit to optical and radar astrometry data

@show success, δx_OR7, Γ_OR7 = newtonls(res, w, zeros(get_numvars()), 10)
x_OR7 = sol_fwd(sol_fwd.t0)(δx_OR7);

In [ ]:
### Perform 6-DOF orbital fit (i.e., without Yarkovsky effect) to optical and radar astrometry data

δx_OR6, Γ_OR6 = newtonls_6v(res, w, zeros(get_numvars()), 10)
x_OR6 = sol_fwd(sol_fwd.t0)(δx_OR6);

In [ ]:
### Print results

# orbital fit
@show success
@show x_OR7 # cartesian state vector at solution reference epoch
@show sqrt.(diag(Γ_OR7)).*vcat(1e-8ones(6),1e-14) # uncertainties at solution reference epoch (1-sigma)
;

# Fit quality statistics: $\chi^2$, normalized RMS

The results below correspond to Table 1 of Pérez-Hernández and Benet (2022).

In [ ]:
# post-fit statistics

nradec = length(res_radec)
res_α = view(res_radec, 1:nradec÷2)
res_δ = view(res_radec, 1+nradec÷2:nradec)
res_τ = res_del
res_ν = res_dop
w_α = view(w_radec, 1:nradec÷2)
w_δ = view(w_radec, 1+nradec÷2:nradec)
res_α_OR6 = res_α(δx_OR6)
res_δ_OR6 = res_δ(δx_OR6)
res_α_OR7 = res_α(δx_OR7)
res_δ_OR7 = res_δ(δx_OR7)

@show nrms_radec = nrms(res_radec(δx_OR7),w_radec)
@show nrms_radec = nrms(vcat(res_del,res_dop)(δx_OR7),vcat(w_del,w_dop))
@show nrms_optrad = nrms(res(δx_OR7),w)
@show mean_ra = mean(res_ra(δx_OR7), weights(w_ra))
@show mean_dec = mean(res_dec(δx_OR7), weights(w_dec))
@show mean_del = mean(res_del(δx_OR7), weights(w_del))
@show mean_dop = mean(res_dop(δx_OR7), weights(w_dop))
@show chi2_optrad = chi2(res(δx_OR7),w)
;

# Semimajor axis drift

In [ ]:
jd0 = J2000 + sol_fwd.t0 #Julian date (TDB) of integration initial time
et0 = (jd0 - J2000)*daysec

In [ ]:
# Apophis heliocentric position and velocity at solution reference epoch (plus autodiff perturbations)
xas = sol_fwd(sol_fwd.t0)[1:6] .- eph_su(jd0-J2000)

In [ ]:
# Orbital elements of new solution (ecliptic frame at J2000.0 epoch)
pv2kep(xas(δx_OR7), jd=jd0, frame=:ecliptic)

In [ ]:
# JPL solution #216 for Apophis
kep_JPL216 = [
    0.1914347568286823, 
    0.7460634895090179, 
    2459424.558511739019, 
    203.9662692226386, 
    126.5900238507279, 
    3.338874353538208, 
    -2.901085583204654E-14, 
    5.E-13
]

In [ ]:
kep_JT = pv2kep(xas, jd=jd0, frame=:ecliptic) # Orbital elements of JT solution at ref. epoch
kep_OR7 = pv2kep(xas(δx_OR7), jd=jd0, frame=:ecliptic) # Orbital elements of new solution at ref. epoch

In [ ]:
# Jacobian of transformation: cartesian state -> Keplerian elements
t_car2kep(x) = hcat(
    TaylorSeries.gradient(kep_JT.e), 
    TaylorSeries.gradient(kep_JT.q),
    TaylorSeries.gradient(kep_JT.tp),
    TaylorSeries.gradient(kep_JT.Ω),
    TaylorSeries.gradient(kep_JT.ω),
    TaylorSeries.gradient(kep_JT.i),
    TaylorSeries.gradient(kep_JT.M),
    TaylorSeries.gradient(kep_JT.a),
    )(x)

# semimajor axis drift \dot{a} due to Yarkovsky effect in units of au per day, as a function of initial condition
ady_kep = yarkp2adot(sol_fwd(sol_fwd.t0)[7], kep_JT.a, kep_JT.e)

# semimajor axis drift, au per million years
ady_kep_au_My = 1e6yr*ady_kep

# semimajor axis drift, meters per year
ady_kep_m_y = 1e3au*yr*ady_kep

# Jacobian of transformation: cartesian state -> semimajor axis drift in au per million years
t_kep2ady_au_My(x) = TaylorSeries.gradient(ady_kep_au_My)(x)

# Jacobian of transformation: cartesian state -> semimajor axis drift in meters per year
t_kep2ady_m_y(x) = TaylorSeries.gradient(ady_kep_m_y)(x)

In [ ]:
# mean semimajor axis drift, au per million years
ady_kep_au_My_OR7 = ady_kep_au_My(δx_OR7) # au/Myr

In [ ]:
# mean semimajor axis drift, meters per year
ady_kep_m_y_OR7 = 1e3au*yr*ady_kep(δx_OR7) # m/yr

In [ ]:
# uncertainty in semimajor axis drift, au per million years
sqrt( t_kep2ady_m_y(δx_OR7)'*Γ*t_kep2ady_m_y(δx_OR7) )

In [ ]:
# uncertainty in semimajor axis drift, au per million years
sqrt( t_kep2ady_au_My(δx_OR7)'*Γ*t_kep2ady_au_My(δx_OR7) )

In [ ]:
# covariance matrix of Keplerian elements
Γ_kep = t_car2kep(δx_OR7)'*Γ*t_car2kep(δx_OR7)

In [ ]:
#formal uncertainties in Keplerian elements (1-sigma)

sqrt.(diag(Γ_kep))

# Plots: optical astrometry post-fit residuals

In [ ]:
#bins_hist = -2.0:0.1:2.0
nbins_plot = 100

#h_a = fit(Histogram, res_α_OR6, bins_hist)
h_a = fit(Histogram, res_α_OR6, nbins=nbins_plot)
bc_a = midpoints(h_a.edges[1]) # bin centers
bs_a = fill(h_a.edges[1].step.hi, length(bc_a)) # bin sizes
;

#h_b = fit(Histogram, res_δ_OR6, bins_hist)
h_b = fit(Histogram, res_δ_OR6, nbins=nbins_plot)
bc_b = midpoints(h_b.edges[1]) # bin centers
bs_b = fill(h_b.edges[1].step.hi, length(bc_b)) # bin sizes

#h_c = fit(Histogram, res_α_OR7, bins_hist)
h_c = fit(Histogram, res_α_OR7, nbins=nbins_plot)
bc_c = midpoints(h_c.edges[1]) # bin centers
bs_c = fill(h_c.edges[1].step.hi, length(bc_c)) # bin sizes

#h_d = fit(Histogram, res_δ_OR7, bins_hist)
h_d = fit(Histogram, res_δ_OR7, nbins=nbins_plot)
bc_d = midpoints(h_d.edges[1]) # bin centers
bs_d = fill(h_d.edges[1].step.hi, length(bc_d)) # bin sizes
;

In [ ]:
minimum(res_α_OR6), maximum(res_α_OR6)

In [ ]:
minimum(res_δ_OR6), maximum(res_δ_OR6)

In [ ]:
minimum(res_α_OR7), maximum(res_α_OR7)

In [ ]:
minimum(res_δ_OR7), maximum(res_δ_OR7)

In [ ]:
mean(res_α_OR6), mean(res_δ_OR6)

In [ ]:
mean(res_α_OR7), mean(res_δ_OR7)

In [ ]:
angv = vcat(range(0, length=1000, stop=prevfloat(2pi)), 0)
sigma_factor = 3.0
σ_cos_ang = sigma_factor*cos.(angv)
σ_sin_ang = sigma_factor*sin.(angv)

cov_αδ_cholf_OR6 = cholesky(cov(hcat(res_α_OR6, res_δ_OR6)))
cov_αδ_cholf_OR7 = cholesky(cov(hcat(res_α_OR7, res_δ_OR7)))

ell_3σ_x_OR6 = mean(res_α_OR6) .+ cov_αδ_cholf_OR6.L[1,1].*σ_cos_ang .+ cov_αδ_cholf_OR6.L[1,2].*σ_sin_ang
ell_3σ_y_OR6 = mean(res_δ_OR6) .+ cov_αδ_cholf_OR6.L[2,1].*σ_cos_ang .+ cov_αδ_cholf_OR6.L[2,2].*σ_sin_ang

ell_3σ_x_OR7 = mean(res_α_OR7) .+ cov_αδ_cholf_OR7.L[1,1].*σ_cos_ang .+ cov_αδ_cholf_OR7.L[1,2].*σ_sin_ang
ell_3σ_y_OR7 = mean(res_δ_OR7) .+ cov_αδ_cholf_OR7.L[2,1].*σ_cos_ang .+ cov_αδ_cholf_OR7.L[2,2].*σ_sin_ang
;

In [ ]:
gap = 0.01
xlim_plt = 3.5
ymin_plt = -5.0
ymax_plt = 5.0

Gaston.set(preamble="set grid; set style circle radius 0.02")

p1 = Gaston.plot(
    bc_a,
    h_a.weights./2, 
    supp=[bs_a./2 h_a.weights./2], 
    curveconf="w boxxy notit fs transparent solid 0.4", 
    Axes(
        #linetype = :tab10,
        axesconf="set lmargin at screen 0.1; set rmargin at screen 0.85; set tmargin at screen 0.99; set bmargin at screen 0.8+$gap",
        xrange = (-xlim_plt, xlim_plt),
        format = "''",
        label="1 '(b)' at graph 0.05,0.8"
    ),
    handle = 1
)
Gaston.plot!(
    bc_c,
    h_c.weights./2, 
    supp=[bs_c./2 h_c.weights./2], 
    curveconf="w boxxy notit fs transparent solid 0.4"
)

p2 = Gaston.plot(
    res_α_OR6, 
    res_δ_OR6, 
    curveconf="w circles fill transparent solid 0.4 noborder tit 'gravity-only'",
    Axes(
        #linetype = :tab10,
        key="at screen 0.835, screen 0.78 box spacing 1.2 height 0.4",  #"outside top right box spacing 1.2 height 1",
        axesconf="set lmargin at screen 0.1; set rmargin at screen 0.85; set tmargin at screen 0.8; set bmargin at screen 0.1",
        xrange = (-xlim_plt, xlim_plt),
        yrange = (ymin_plt, ymax_plt),
        format = "\"%g\"",
        xlabel="'\$\\alpha\\cos(\\delta)\$ \$[\$arcsec\$]\$'",
        ylabel="'\$\\delta\$ \$[\$arcsec\$]\$'",
        label="1 '(a)' at graph 0.05,0.925"
    ), 
    handle = 2
)
Gaston.plot!(res_α_OR7, res_δ_OR7, 
    curveconf="w circles fill transparent solid 0.4 noborder tit 'non-grav.'"
)
Gaston.plot!([mean(res_α_OR6)], [mean(res_δ_OR6)], curveconf="w p pt 4 ps 1.5 lw 8") # tit 'mean R.A./Dec (OR6)'
Gaston.plot!([mean(res_α_OR7)], [mean(res_δ_OR7)], curveconf="w p pt 6 ps 1.5 lw 8") # tit 'mean R.A./Dec (OR7)'
Gaston.plot!(ell_3σ_x_OR6, ell_3σ_y_OR6, curveconf="w l lw 8 lc 3")
Gaston.plot!(ell_3σ_x_OR7, ell_3σ_y_OR7, curveconf="w l lw 8 lc 4")

p4 = Gaston.plot(
    h_b.weights./2,
    bc_b,
    supp=[h_b.weights./2 bs_b./2],
    curveconf="w boxxy notit fs transparent solid 0.4",
    Axes(
        #linetype = :tab10,
        axesconf="set lmargin at screen 0.85+$gap; set rmargin at screen 0.99; set tmargin at screen 0.8; set bmargin at screen 0.1",
        yrange = (ymin_plt, ymax_plt),
        format = "''",
        label="1 '(c)' at graph 0.15,0.925"
    ),
    handle = 4
)
Gaston.plot!(
    h_d.weights./2,
    bc_d,
    supp=[h_d.weights./2 bs_d./2],
    curveconf="w boxxy notit fs transparent solid 0.4"
)

Gaston.plot([p1 nothing ; p2 p4])

#Gaston.save(term = "cairolatex", output = "radec_hist.tex", saveopts = "pdf standalone color dashed transparent size 7in,5in font ',12'")
#run(`lualatex radec_hist.tex`)

# Plots: radar astrometry post-fit residuals

In [ ]:
del_index = findall(x->x.Δτ_units=="us", deldop)
dop_index = findall(x->x.Δν_units=="Hz", deldop)
length(del_index), length(dop_index)

In [ ]:
length(deldop)

In [ ]:
del_dates_plot_ = date.(deldop[del_index])
dop_dates_plot_ = date.(deldop[dop_index])

del_dates_plot = Dates.format.(del_dates_plot_, "dd/mm/yy HHMM")
dop_dates_plot = Dates.format.(dop_dates_plot_, "dd/mm/yy HHMM")

σ_del_all = delay_sigma.(deldop[del_index])
σ_dop_all = doppler_sigma.(deldop[dop_index])
;

In [ ]:
p1 = Gaston.plot(
    del_index,
    res_τ(δx_OR6),
    supp = σ_del_all,
    key="box spacing 1.2 height 0.4", #at screen 0.835, screen 0.78 
    curveconf = "u 1:2:3 w errorbars pt 6 lw 4 tit 'gravity-only'",
    Axes(
        #axesconf="set lmargin at screen 0.15; set rmargin at screen 0.95; set bmargin at screen 0.5",
        axesconf="set lmargin at screen 0.11",
        #linetype = :tab10,
        bars = 2,
        ylabel = "'Time-delay resid. \$[\$us\$]\$'",
        xrange = (0,39),
        xtics = "1,2,40",
        format = "y '%1.1f'",
        yrange=(-7,8),
        label="1 '(a)' at graph 0.05,0.9"#,
        #title = "'gravity-only'"
    ),
    handle = 1
)
Gaston.plot!(
    del_index,
    res_τ(δx_OR7),
    supp = σ_del_all,
    curveconf = "u 1:2:3 w errorbars pt 4 lw 4 tit 'non-gravitational'",
    lc = 2
)

p2 = Gaston.plot(
    dop_index,
    res_ν(δx_OR6),
    supp = σ_dop_all,
    curveconf = "u 1:2:3 w errorbars pt 6 lw 4 tit 'gravity-only'",
    key="box spacing 1.2 height 0.4",
    Axes(
        #axesconf="set lmargin at screen 0.15; set rmargin at screen 0.95; set tmargin at screen 0.5",
        axesconf="set lmargin at screen 0.11",
        #linetype = :tab10,
        bars = 2,
        ylabel = "'Doppler shift resid. \$[\$Hz\$]\$'",
        xrange = (0,39),
        xtics = "1,2,40",
        format = "y '%1.1f'",
        yrange=(-0.6,0.8),
        label="1 '(b)' at graph 0.05,0.9" #,
        # title = "'non-gravitational'"
    ),
    handle = 2
)
Gaston.plot!(
    dop_index,
    res_ν(δx_OR7),
    supp = σ_dop_all,
    curveconf = "u 1:2:3 w errorbars pt 4 lw 4 tit 'non-gravitational'",
    lc = 2
)


Gaston.plot([p1; p2])

#Gaston.save(term = "cairolatex", output = "deldop_residuals.tex", saveopts = "pdf standalone color dashed transparent size 10in,7in font ',14'")
#run(`pdflatex deldop_residuals.tex`)

In [ ]:
deldop_dates_plot_xaxis = vcat("01/06/04 0000", del_dates_plot, "31/03/21 2359");

In [ ]:
tmargin_bmargin_gap = 0.03
bmargin_ths_plt = 0.5 + tmargin_bmargin_gap
tmargin_ths_plt = 0.965
axsbrk_v = 1.25*0.01
axsbrk_h = 1.25*0.005
lw_ths_plt = 3
rmargin_ths_plt = 0.3

p1 = Gaston.plot(
    del_dates_plot,
    res_τ(δx_OR6),
    supp = σ_del_all,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt",
    Axes(
        #linetype = :tab10,
        axesconf = "set rmargin at screen 0.3; set tmargin at screen $tmargin_ths_plt; set bmargin at screen $bmargin_ths_plt",
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        xtics = false, #"format '' time (\"01/01/05\",\"01/01/06\")", # '%d/%b/%y'
        xrange = "[\"01/12/04\":\"30/05/06\"]",
        #xlabel  = "\"Date\"",
        bars = false, #2,
        ylabel = "'Time-delay resid. (O-C) [\$\\mu\$s]'",
        border = "1+2+4",
        ytics = ["nomirror", "format '%1.1f'"],
        yrange = "[-8.0:6.0]",
        arrow = [
            "from screen 0.3-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.3+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.3-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.3+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead"
            ],
        label="1 '(a)' at graph 0.2,0.9",
        grid=false
        #arrow = "from screen 0.3-0.01,1.0-0.01 to screen 0.3+0.01,1.0+0.01 nohead"
        #format = "x ''"
        #title = "'Time-delay resid. (O-C)'"
    ),
    handle = 1
)
Gaston.plot!(
    del_dates_plot,
    res_τ(δx_OR7),
    supp = σ_del_all,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2")

p2 = Gaston.plot(
    del_dates_plot,
    res_τ(δx_OR6),
    supp = σ_del_all,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt",
    Axes(
        #linetype = :tab10,
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        axesconf = "set lmargin at screen 0.35; set tmargin at screen $tmargin_ths_plt; set rmargin at screen 0.8; set bmargin at screen $bmargin_ths_plt",
        xtics = false, #"format '' time (\"01/01/13\",\"01/02/13\",\"01/03/13\")", # '%d/%b/%y'
        xrange = "[\"15/12/12\":\"20/03/13\"]",
        #xlabel  = "\"Date\"",
        bars = false, #2,
        #tics="scale 0",
        ytics = false, #[false, "(\"\" 0)"], #"nomirror",
        border = "1+4",
        yrange = "[-8.0:6.0]",
        format = ["x ''", "y ''"], #%1.1f
        arrow = [
            "from screen 0.35-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.35+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.35-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.35+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.80-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.80+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.80-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.80+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead"
            ],
        #ylabel = "'[us]'"
    ),
    handle = 2
)
Gaston.plot!(
    del_dates_plot,
    res_τ(δx_OR7),
    supp = σ_del_all,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2")

p3 = Gaston.plot(
    del_dates_plot,
    res_τ(δx_OR6),
    supp = σ_del_all,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt",
    Axes(
        #linetype = :tab10,
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        axesconf = "set lmargin at screen 0.85; set tmargin at screen 0.965; set bmargin at screen $bmargin_ths_plt",
        xtics = false, #"format '' time (\"03/03/21\",\"11/03/21\")", #'%d/%b/%y'
        xrange = "[\"02/03/21\":\"15/03/21\"]",
        #xlabel = "\"Date\"",
        bars = false, #2,
        #tics="scale 0",
        ytics = false, #[false, "(\"\" 0)"], #"nomirror",
        border = "1+4+8",
        yrange = "[-8.0:6.0]",
        format = ["x ''", "y ''"], #%1.1f
        arrow = [
            "from screen 0.85-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.85+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.85-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.85+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead"
            ],
        #ylabel = "'[us]'"
    ),
    handle = 3
)
Gaston.plot!(
    del_dates_plot,
    res_τ(δx_OR7),
    supp = σ_del_all,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2")

Gaston.plot([p1 p2 p3])

#Gaston.save(term = "cairolatex", output = "del_resids.tex", saveopts = "pdf standalone color dashed transparent size 10in,7in font ',14'")
#run(`pdflatex del_resids.tex`)

In [ ]:
tmargin_ths_plt = 0.5 - tmargin_bmargin_gap
bmargin_ths_plt = 0.05

Gaston.set(preamble="unset grid")
p4 = Gaston.plot(
    dop_dates_plot,
    res_ν(δx_OR6),
    supp = σ_dop_all,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt",
    Axes(
        #linetype = :tab10,
        axesconf="set rmargin at screen $rmargin_ths_plt; set tmargin at screen $tmargin_ths_plt; set bmargin at screen $bmargin_ths_plt",
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        xtics = "format '%d/%b/%y' time (\"01/01/05\",\"01/01/06\")",
        xrange = "[\"01/12/04\":\"30/05/06\"]",
        #xlabel  = "\"Date\"",
        bars = false,#2,
        ylabel = "'Doppler shift resid. (O-C) [Hz]'",
        border="1+2+4",
        ytics = ["nomirror", "format '%1.1f'"],
        yrange="[-0.5:0.5]",
        arrow = [
            "from screen 0.3-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.3+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.3-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.3+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead"
            ],
        label="1 '(b)' at graph 0.2,0.9"
        #title = "'Doppler shift resid. (O-C)'"
    ),
    handle = 4
)
Gaston.plot!(
    dop_dates_plot,
    res_ν(δx_OR7),
    supp = σ_dop_all,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2")

p5 = Gaston.plot(
    dop_dates_plot,
    res_ν(δx_OR6),
    supp = σ_dop_all,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt",
    Axes(
        #linetype = :tab10,
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        axesconf="set lmargin at screen $rmargin_ths_plt+0.05; set rmargin at screen 0.8; set tmargin at screen $tmargin_ths_plt; set bmargin at screen $bmargin_ths_plt",
        xtics="format '%d/%b/%y' time (\"01/01/13\",\"01/02/13\",\"01/03/13\")",
        xrange="[\"15/12/12\":\"20/03/13\"]",
        #xlabel  = "\"Date\"",
        bars = false, #2,
        ytics=false,
        border="1+4",
        yrange="[-0.5:0.5]",
        arrow = [
            "from screen 0.35-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.35+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.35-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.35+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.8-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.8+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.8-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.8+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead"
            ],
        #ylabel = "'[Hz]'"
    ),
    handle = 5
)
Gaston.plot!(
    dop_dates_plot,
    res_ν(δx_OR7),
    supp = σ_dop_all,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2")

p6 = Gaston.plot(
    dop_dates_plot,
    res_ν(δx_OR6),
    supp = σ_dop_all,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt",
    Axes(
        #linetype = :tab10,
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        axesconf="set lmargin at screen 0.85; set tmargin at screen $tmargin_ths_plt; set bmargin at screen $bmargin_ths_plt",
        xtics="format '%d/%b/%y' time (\"03/03/21\",\"13/03/21\")",
        xrange="[\"02/03/21\":\"15/03/21\"]",
        #xlabel  = "\"Date\"",
        bars = false, #2,
        #format="y ''",
        ytics=false,
        border="1+4+8",
        yrange="[-0.5:0.5]",
        arrow = [
            "from screen 0.85-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.85+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.85-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.85+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead"
            ],
        #ylabel = "'[Hz]'"
    ),
    handle = 6
)
Gaston.plot!(
    dop_dates_plot,
    res_ν(δx_OR7),
    supp = σ_dop_all,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2")

Gaston.plot([p4 p5 p6])

#Gaston.save(term = "cairolatex", output = "dop_resids.tex", saveopts = "pdf standalone color dashed transparent size 10in,7in font ',14'")
#run(`pdflatex dop_resids.tex`)

In [ ]:
Gaston.plot([p1 p2 p3; p4 p5 p6])

#Gaston.save(term = "cairolatex", output = "deldop_residuals_dates.tex", saveopts = "pdf standalone color dashed transparent size 10in,7in font ',14'")
#run(`lualatex deldop_residuals_dates.tex`)